In [28]:
import numpy as np
from joblib import Parallel, delayed
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import math
from multiprocessing import Pool, cpu_count
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [29]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None, var_red=None, variance=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.var_red = var_red  
        self.variance = variance 

class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=2,n_features=None):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features=n_features

    def build_tree(self, dataset, curr_depth=0, parent_variance=None):
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)
        if not self.n_features:
            self.n_features = math.sqrt(num_features)
        
        if parent_variance is None:
            parent_variance = np.var(Y)
        
        best_split = {}
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            feature_indices = np.random.choice(num_features, self.n_features, replace=False)
            best_split = self.get_best_split(dataset, num_samples, feature_indices, parent_variance)
            if best_split["var_red"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1, best_split["left_variance"])
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1, best_split["right_variance"])
                return Node(
                    feature_index=best_split["feature_index"], 
                    threshold=best_split["threshold"], 
                    left=left_subtree, 
                    right=right_subtree, 
                    var_red=best_split["var_red"],
                    variance=parent_variance
                )
        
        # Create a leaf node if no valid split is found
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value, variance=parent_variance)

    def get_best_split(self, dataset, num_samples, feature_indices, parent_variance):
        best_split = {
            "feature_index": None,
            "threshold": None,
            "dataset_left": None,
            "dataset_right": None,
            "var_red": -float("inf"),
            "left_variance": None,
            "right_variance": None
        }

        y = dataset[:, -1]
        total_samples = len(y)
        
        for feature_index in feature_indices:
            feature_values = dataset[:, feature_index]
            
            sorted_indices = np.argsort(feature_values)
            sorted_dataset = dataset[sorted_indices]
            sorted_y = y[sorted_indices]

            prefix_sum = np.cumsum(sorted_y)
            prefix_sq_sum = np.cumsum(sorted_y ** 2)

            for i in range(1, num_samples):
                if feature_values[sorted_indices[i]] == feature_values[sorted_indices[i - 1]]:
                    continue  

                left_count = i
                right_count = num_samples - i
                left_sum = prefix_sum[i - 1]
                left_sq_sum = prefix_sq_sum[i - 1]
                right_sum = prefix_sum[-1] - left_sum
                right_sq_sum = prefix_sq_sum[-1] - left_sq_sum

                left_var = (left_sq_sum / left_count) - (left_sum / left_count) ** 2 if left_count > 0 else 0
                right_var = (right_sq_sum / right_count) - (right_sum / right_count) ** 2 if right_count > 0 else 0

                var_red = parent_variance - (
                    (left_count / total_samples) * left_var
                    + (right_count / total_samples) * right_var
                )

                if var_red > best_split["var_red"]:
                    best_split = {
                        "feature_index": feature_index,
                        "threshold": feature_values[sorted_indices[i]],
                        "dataset_left": sorted_dataset[:i],
                        "dataset_right": sorted_dataset[i:],
                        "var_red": var_red,
                        "left_variance": left_var,
                        "right_variance": right_var
                    }

        return best_split


    def calculate_leaf_value(self, Y):
        return np.mean(Y)
    
    def fit(self, X, Y):
        if len(X) == 0 or len(Y) == 0:
            raise ValueError("Input data cannot be empty.")
        dataset = np.concatenate((X, Y.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        return [self.make_prediction(x, self.root) for x in X]

In [30]:
import numpy as np
import math

class RandomForestRegressor:
    def __init__(self, n_trees=10, max_depth=5, min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, Y, X_val=None, Y_val=None):
        n_samples, n_features = X.shape
        if self.n_features is None:
            self.n_features = int(math.sqrt(n_features))  # Default: sqrt of total features
        
        for i in range(self.n_trees):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_sample, Y_sample = X[indices], Y[indices]
            
            tree = DecisionTreeRegressor(
                min_samples_split=self.min_samples_split, 
                max_depth=self.max_depth, 
                n_features=self.n_features
            )
            tree.fit(X_sample, Y_sample)
            self.trees.append(tree)
            
            if X_val is not None and Y_val is not None and (i + 1) % 5 == 0:
                predictions = self.predict(X_val)
                val_loss = np.mean((Y_val - predictions) ** 2)  # Mean squared error
                print(f"Iteration {i + 1}/{self.n_trees}: Validation Loss = {val_loss:.4f}")

    def predict(self, X):
        tree_predictions = np.array([tree.predict(X) for tree in self.trees])
        return np.mean(tree_predictions, axis=0)


In [ ]:
train_path = 'SP_Train.xlsx'  
train_data = pd.read_excel(train_path)

train_data['Item_Weight'].fillna(train_data['Item_Weight'].mean(), inplace=True)
train_data.loc[
    (train_data['Outlet_Type'] == 'Grocery Store') & (train_data['Outlet_Size'].isna()),
    'Outlet_Size'
] = 'Small'

train_data.loc[
    (train_data['Outlet_Type'] == 'Supermarket Type1') & (train_data['Outlet_Size'].isna()),
    'Outlet_Size'
] = 'Small'

train_data.loc[
    (train_data['Outlet_Type'] == 'Supermarket Type2') & (train_data['Outlet_Size'].isna()),
    'Outlet_Size'
] = 'Medium'

train_data.loc[
    (train_data['Outlet_Type'] == 'Supermarket Type3') & (train_data['Outlet_Size'].isna()),
    'Outlet_Size'
] = 'Medium'

train_data['Item_Fat_Content'] = train_data['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'})

train_data_encoded = pd.get_dummies(train_data, drop_first=True)

X = train_data_encoded.drop(columns=['Item_Outlet_Sales'])
y = np.array(train_data_encoded['Item_Outlet_Sales'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=1326, svd_solver='randomized', random_state=42)
X_pca = pca.fit_transform(X_scaled)
print(X_pca.shape)

X_train, X_val, y_train, y_val = train_test_split(X_pca, y, test_size=0.2, random_state=42)

def evaluate_model(n_trees, max_depth, n_features):
    rf = RandomForestRegressor(
        n_trees=n_trees, max_depth=max_depth, n_features=n_features
    )
    rf.fit(X_train, y_train, X_val, y_val) 
    y_pred = rf.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    return mse

n_trees_list = [200]
max_depth_list = [3]
n_features_list = [None]  # None means using root of total features

best_mse = float('inf')
best_params = {}

for n_trees in n_trees_list:
    for max_depth in max_depth_list:
        for n_features in n_features_list:
            print(f"Evaluating: n_trees={n_trees}, max_depth={max_depth}, n_features={n_features}")
            mse = evaluate_model(n_trees, max_depth, n_features)
            print(f"Mean Squared Error: {mse}")
            if mse < best_mse:
                best_mse = mse
                best_params = {'n_trees': n_trees, 'max_depth': max_depth, 'n_features': n_features}

print("\nBest Hyperparameters:")
print(f"n_trees: {best_params['n_trees']}")
print(f"max_depth: {best_params['max_depth']}")
print(f"n_features: {best_params['n_features']}")
print(f"Best Mean Squared Error: {best_mse}")

/var/folders/g4/73wglc1561dbz4ljhlw_hlc80000gn/T/ipykernel_46671/1201578366.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Item_Weight'].fillna(train_data['Item_Weight'].mean(), inplace=True)


(7998, 1326)
